In [ ]:
!git clone --quiet https://github.com/tensorflow/models.git tf_models
  
!cp -r tf_models/research/object_detection object_detection/

!cp -r tf_models/research/slim slim/

!cp tfd/model_main.py object_detection/model_main.py

!rm -rf tfd

!rm -rf tf_models

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

!git clone https://github.com/cocodataset/cocoapi.git

%cd cocoapi/PythonAPI

!make

%cd /content

!cp -r cocoapi/PythonAPI/pycocotools pycocotools/

!rm -rf cocoapi

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content:/content/slim


In [ ]:
!pip install tensorflow-gpu==1.14
!pip install tensorflow==1.15
!pip install keras==2.0.8
!pip install Pillow==2.2.1
!pip install h5py==2.6.0

In [ ]:
!pip install tf_slim
!python object_detection/builders/model_builder_test.py

In [ ]:
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

train_file_id = '1orq0y-8GtfOWl1tBko03rSZT7b3sVfBf' 
eval_file_id = '18nLlxkdJtwfbOaFvpdLhJXrknfzwNNKw'
pbtxt_file_id = '1Dgz6t1fTvhNaxrmfXaPFB4-BYG6XVVXd'
config_file_id = '15PYisbZr1tzcEJQtI6Z-qtlhWZysHIuP'


data_dir = 'data'
config_dir = 'config'

if not os.path.isdir(data_dir):
  os.makedirs(data_dir)

downloaded = drive.CreateFile({'id': train_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'train.record'))

downloaded = drive.CreateFile({'id': eval_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'eval.record'))

downloaded = drive.CreateFile({'id': pbtxt_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'udacity_label_map.pbtxt'))

if not os.path.isdir(config_dir):
  os.makedirs(config_dir)

downloaded = drive.CreateFile({'id': config_file_id})
downloaded.GetContentFile(os.path.join(config_dir, 'ssd_inception_v2_coco_udacity.config'))



In [ ]:
import urllib.request
import tarfile

MODEL_NAME =  'ssd_inception_v2_coco_2017_11_17' #'ssd_mobilenet_v1_coco_11_06_2017' #'ssd_mobilenet_v1_coco_2018_01_28'  # use this in case the other is incompatible w/ TF 1.3
MODEL_FILE = MODEL_NAME + '.tar.gz'
MODEL_URL = 'http://download.tensorflow.org/models/object_detection/' + MODEL_FILE

MODELS_DIR = 'models'

if not os.path.isdir(MODELS_DIR):
  os.makedirs(MODELS_DIR)

if not os.path.exists(MODEL_FILE):
  urllib.request.urlretrieve(MODEL_URL, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
os.rename(MODEL_NAME, os.path.join(MODELS_DIR, MODEL_NAME))

In [ ]:
FINE_TUNED_DIR = os.path.join(MODELS_DIR, 'fine_tuned')

if not os.path.isdir(FINE_TUNED_DIR):
  os.makedirs(FINE_TUNED_DIR)

!python object_detection/model_main.py \
        --pipeline_config_path=config/ssd_inception_v2_coco_udacity.config \
        --model_dir=models/fine_tuned/ssd_inception_v2_coco_udacity \


In [ ]:
# To load tensorboard and plot the graphs
%load_ext tensorboard
%tensorboard --logdir='models/fine_tuned/ssd_inception_v2_coco_udacity'

In [ ]:
## To download the trained models 
# !zip -r models/fine_tuned/ssd_inception_v2_coco_udacity.zip models/fine_tuned/ssd_inception_v2_coco_udacity
# from google.colab import files
# files.download("models/fine_tuned/ssd_inception_v2_coco_udacity.zip")
# files.download("models/fine_tuned/ssd_mobilenet/model.ckpt")

In [ ]:
# Replace xxxx with the last saved checkpoint and export the frozen inference graph
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path config/ssd_inception_v2_coco_udacity.config --trained_checkpoint_prefix models/fine_tuned/ssd_inception_v2_coco_udacity/model.ckpt-xxxx --output_directory models/checkpoint